In [29]:
import json
import os

import numpy as np
from datetime import datetime,date
from utils.gcp import GoogleSheetsClient
import pandas as pd
import json
import numpy as np



### Bigquery client

The content of the gcp.json file must be replaced by the file sent in the mail. This file must contain the key of the service account.

In [30]:
from google.cloud import bigquery
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='utils/gcp.json'

with open(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')) as f:
   secret = json.load(f)

client = bigquery.Client()
cliente_sheet=GoogleSheetsClient(secret)

In [31]:
def execute_response(client,query):
    rows=client.query(query).result()
    df=[dict(row) for row in rows]
    return df

### Data Manipulation And Cleaning
Read .csv with Pandas, clean date formats, and replace the NAN values ​​with None, so that they are empty in the table

#### Loan Data

In [33]:
loan=pd.read_csv('files/loan_data.csv')
loan.sample(5)




,customer_id,loan_id,application_date,term,disbursed_date,loanamount,paid_date,is_disbursed
11617,61z2529z0h37hzyg296hy295,loan_0317522,27/11/21 10:00,NaN,NaN,7299.2700,NaN,False
710,5y33961839754g6y5yi976g2,loan_8139574,07/01/22 19:26,6.0,NaN,862.8600,NaN,False
11851,619hg6yz9g5176943892531i,loan_6997123,23/11/21 21:51,2.0,23/11/21,2608.3265,29/12/21,True
2586,61yi3y4yg592g9x253y243i7,loan_0991957,15/03/22 18:20,4.0,15/03/22,698.6100,NaN,True
11614,617g16i4i14y6458y8hh504y,loan_5319295,01/02/22 23:45,NaN,NaN,9343.2700,NaN,False


In [34]:
loan['application_date']=pd.to_datetime(loan['application_date'], format='%d/%m/%y %H:%M').dt.strftime("%Y/%m/%d %H:%M")
loan['disbursed_date']=pd.to_datetime(loan['disbursed_date'], format='%d/%m/%y').dt.strftime("%Y/%m/%d")
loan['paid_date']=pd.to_datetime(loan['paid_date'], format='%d/%m/%y').dt.strftime("%Y/%m/%d")
loan.replace({np.nan: None}, inplace = True)
loan.sample(5)

,customer_id,loan_id,application_date,term,disbursed_date,loanamount,paid_date,is_disbursed
12385,61g674hhg3874i4yz6y16z2z,loan_5963323,2022/01/27 01:03,8.0,2022/01/27,4253.71,None,True
10098,6176ziziig1hh684hg1x9574,loan_0095691,2022/03/29 19:42,8.0,2022/03/30,959.22,None,True
12756,6188y3xi8i3i4zy95x06i20y,loan_4453860,2021/11/10 19:02,8.0,2021/11/10,6204.27,None,True
290,617g4888761zhggy2yh537z0,loan_2630434,2021/11/15 17:08,None,None,656.343,None,False
11310,61yg947x9i49h8ghg33ixg6z,loan_4743822,2022/03/31 22:30,None,None,290.54,None,False


### Repayment Data

In [ ]:
repayment=pd.read_csv('files/repayment_data.csv')
repayment.sample(5)

In [17]:


repayment['installment_duedate']=pd.to_datetime(repayment['installment_duedate'], format='%Y-%m-%d').dt.strftime("%Y-%m-%d")
repayment['repaid_date']=pd.to_datetime(repayment['repaid_date'], format='%Y-%m-%d').dt.strftime("%Y-%m-%d")
repayment.replace({np.nan: None}, inplace = True)

repayment.sample(5)


## Export data to google sheets
In bigquery there are two tables that are fed from the gsheets that I export:
- `sofia-fasce.STG.BT_LOAN_DATA`
- `sofia-fasce.STG.BT_REPAYMENT_DATA` 

The initial idea was to perform the data update directly from python without going through gsheet. But in the trial version this is not available


In [19]:
sheets_id_loan='10v9266PP-kRDvJLucrrhdGv5U1NWrzFr_gC9ASz747c'
sheet_id_repayment='1ypPjTFlYmywHATaznm9E8oCrDeSA_cGc7wL3PpDDors'

Clean the gsheet files if we want to reprocess.

In [38]:
cliente_sheet.delete_columns(sheets_id_loan, 'loan_data', 0,len(loan.columns))
cliente_sheet.delete_columns(sheet_id_repayment, 'repayment_data', 0,len(repayment.columns))

{'spreadsheetId': '1ypPjTFlYmywHATaznm9E8oCrDeSA_cGc7wL3PpDDors',
 'replies': [{}]}

Upload dataframes in Gsheets. The tables will be automatically fed with the new information


In [39]:

cliente_sheet.upload_dataframe_to_spreadsheet(
    loan, sheets_id_loan, tab_name=f"loan_data", start_cell="A1"
)

{'spreadsheetId': '10v9266PP-kRDvJLucrrhdGv5U1NWrzFr_gC9ASz747c',
 'updates': {'spreadsheetId': '10v9266PP-kRDvJLucrrhdGv5U1NWrzFr_gC9ASz747c',
  'updatedRange': 'loan_data!A1:H13168',
  'updatedRows': 13168,
  'updatedColumns': 8,
  'updatedCells': 78422}}

In [40]:

cliente_sheet.upload_dataframe_to_spreadsheet(
    repayment, sheet_id_repayment, tab_name="repayment_data", start_cell="A1"
)

{'spreadsheetId': '1ypPjTFlYmywHATaznm9E8oCrDeSA_cGc7wL3PpDDors',
 'updates': {'spreadsheetId': '1ypPjTFlYmywHATaznm9E8oCrDeSA_cGc7wL3PpDDors',
  'updatedRange': 'repayment_data!A1:H29437',
  'updatedRows': 29437,
  'updatedColumns': 8,
  'updatedCells': 222840}}

If necessary, Grant permissions to the gsheet files.

In [41]:
cliente_sheet.grant_permissions(sheets_id_loan, "writer", 'sofia.fsc.challenge@gmail.com')
cliente_sheet.grant_permissions(sheet_id_repayment, "writer", 'sofia.fsc.challenge@gmail.com')

{'kind': 'drive#permission',
 'id': '15974111209173566003',
 'type': 'user',
 'role': 'owner'}

## Creation PROD tables
Creation of productive tables (or views) defining the data types of the columns.

In [46]:
query_productive_loan_data="""create or replace table `sofia-fasce.PROD.BT_LOAN_DATA` as 
                                SELECT customer_id,loan_id,
                                DATETIME(application_date, 'America/Mexico_City') as application_date,
                                cast(term as int) as term,
                                PARSE_DATE ('%Y/%m/%d',disbursed_date) as disbursed_date,
                                cast(loanamount as numeric) as loanamount ,
                                PARSE_DATE ('%Y/%m/%d',paid_date) as paid_date,
                                cast(is_disbursed as bool) as is_disbursed
                                FROM `sofia-fasce.STG.BT_LOAN_DATA`""" 

client.query(query_productive_loan_data)

In [49]:
query_productive_repayment_data= """create table `sofia-fasce.PROD.BT_REPAYMENT_DATA` as 
                                SELECT 
                                loan_id,	
                                installment_id,
                                cast(installment_number as int) as installment_number,
                                installment_duedate,
                                repaid_date,
                                cast(capital_due as numeric) as capital_due ,
                                cast(capital_paid as numeric) as capital_paid ,
                                cast(interests_paid as numeric) as interests_paid 
                                FROM `sofia-fasce.STG.BT_REPAYMENT_DATA`"""
client.query(query_productive_repayment_data)

## Data manipulation and result table creation
- Definition 1. “recurrence_1” (1,2,3,4,...., n):
The number of loans disbursed in the client's history at the disbursement date,
including the one being disbursed.
- Definition 2. “recurrence_2” (TRUE, FALSE):
recurrent: A loan is said to be recurrent if, at the disbursement date, the customer
has paid any loan in its totality or has paid 3 or more installments of its first loan
disbursed.

In [48]:
query_recurrence="""create table PROD.BT_RECURRENCE AS
      WITH loan_data as (
                        SELECT a.*,
                              ROW_NUMBER() over (partition by customer_id order by disbursed_date) recurrence_1,
                              CASE WHEN 
                                    MIN(paid_date) over (partition by customer_id order by disbursed_date  ROWS UNBOUNDED PRECEDING )
                                    <= disbursed_date 
                              THEN 1 ELSE 0 END as paid_any_loan_last
                        FROM `PROD.BT_LOAN_DATA` a
                        WHERE is_disbursed=true
      ),
      repay_complete_data as (
                        SELECT a.customer_id,
                                a.loan_id,
                                a.term,
                                a.disbursed_date,
                                a.paid_date,
                                a.recurrence_1,
                                a.paid_any_loan_last,
                                b.installment_number,
                                b.repaid_date	
                        FROM loan_data a
                        JOIN `PROD.BT_REPAYMENT_DATA` b on a.loan_id=b.loan_id
      )
      SELECT 
            customer_id,
            loan_id,
            recurrence_1,
            CASE WHEN first_loan_3_or_more_terms=1 or paid_any_loan_last=1 THEN True ELSE False END AS recurrence_2   
      FROM  (
                  SELECT * ,
                  CASE WHEN 
                        MIN( CASE WHEN recurrence_1=1 AND	installment_number>=3  THEN	repaid_date ELSE NULL END) 
                        over(partition by customer_id order by disbursed_date  ROWS UNBOUNDED PRECEDING ) 
                        <=disbursed_date 
                  THEN 1 ELSE 0 END AS first_loan_3_or_more_terms
                  FROM repay_complete_data
            ) GROUP BY 1,2,3,4;

"""
client.query(query_recurrence)

### Checks 

In [54]:
query="""select count(*) ,count(distinct loan_id)
FROM PROD.BT_RECURRENCE"""
df=execute_response(client,query)

In [51]:
df

[{'f0_': 4908, 'f1_': 4908}]

In [52]:
query="""select count(*) ,count(distinct loan_id)
FROM PROD.BT_LOAN_DATA
WHERE is_disbursed=true"""
df=execute_response(client,query)


In [53]:
df

[{'f0_': 4908, 'f1_': 4908}]